# Transaction Utility Theory

This notebook aims to recreate some of the empirical findings of Thaler, R. (1985). Mental accounting and consumer choice. Marketing Science, 4(3), 199-214. 
Specifically we are interested in whether LLMs' responses are similar to the original responses in **section 3** of the paper, regarding the Hockey Ticket Question.

<span style="color:red">
TBD: <br>
Show distribution of all experiment answers, or just $0, $5, $10 and label everything else as "others"?
</span>


---

In [6]:
from openai import OpenAI
import openai
import matplotlib.pyplot as plt
import os 
import numpy as np
import pandas as pd
from tqdm import tqdm
import replicate
from ast import literal_eval
import plotly.graph_objects as go
import pickle

In [7]:
# Get openAI API key (previously saved as environmental variable)
openai.api_key = os.environ["OPENAI_API_KEY"]

# Set client
client = OpenAI()

# Set global plot style
plt.style.use('seaborn-v0_8')

# Set plots to be displayed in notebook
%matplotlib inline

---

#### Setting up prompts for the experiment



- LLMs used in the experiment:
    - GPT-3.5-Turbo         (ID = 1)
    - GPT-4-1106-Preview    (ID = 2)
    - LLama-70b             (ID = 3)

We can differentiate between the following scenario combinations:

- Initial ticket price:
    - free                  (ID = 1)
    - $5 (as on ticket)     (ID = 2)
    - $10                   (ID = 3)
- Current market price:
    - $5                    (ID = 1)
    - $10                   (ID = 2)
- Selling to:
    - Friend                (ID = 1)
    - Stranger              (ID = 2)



Similar to the Prospect Theory and Decoy Effect notebooks, we will use experiment IDs to run the study. The IDs will be constructed as:

*TU_model_initialprice_currentprice_buyer*

Therefore, TU_2_2_1_2 would mean we used GPT-4-1106-Preview, an initial ticket price of $5, a current market price of $5 as well and we are selling to a stranger.

We leave out the information of the respondent being a student in all prompts. From experience, the more concise a prompt is, the better the answer quality. 
Since the job status/education level is not of interest here, we leave out this information in order to formulate clearer prompts. 

In [8]:
# Set up list of initial costs
initial_costs = ["but you were given your tickets for free by a friend.", "which is what you paid for each ticket.", "but you paid $10 each for your tickets when you bought them."]

# Set up list of current ticket prices
orientation_prices = ["$5", "$10"]

# Set up list of potential buyers in a scenario
potential_buyers = ["friend?", "stranger?"]

- Constructing the prompts

In [9]:
TU_prompts = []
for costs in initial_costs:
    for orientation_price in orientation_prices:
        for potential_buyer in potential_buyers:
            prompt = f"""Imagine that you are going to a soldout Cornell hockey playoff game, and you have an extra ticket to sell or give away. The price marked on the ticket is $5 {costs}
            You get to the game early to make sure you get rid of the ticket. An informal survey of people selling tickets indicates that the going price is {orientation_price}. 
            You find someone who wants the ticket and takes out his wallet to pay you. He asks, how much you want for the ticket. 
            Assume that there is no law against charging a price higher than that marked on the ticket. What price do you ask for, if he is a {potential_buyer}"""
            TU_prompts.append(prompt)

In [10]:
# Save prompts for use in Dashboard
with open("Output/TU_prompts.pkl", "wb") as file:
    pickle.dump(TU_prompts, file)

- TU_prompts[0]: free, $5, friend  -> 1_1_1 (Configuration 1)
- TU_prompts[1]: free, $5, stranger -> 1_1_2 (Configuration 2)
- TU_prompts[2]: free, $10, friend -> 1_2_1 (Configuration 3)
- TU_prompts[3]: free, $10, stranger -> 1_2_2 (Configuration 4)
- TU_prompts[4]: $5, $5, friend -> 2_1_1 (Configuration 5)
- TU_prompts[5]: $5, $5, stranger -> 2_1_2 (Configuration 6)
- TU_prompts[6]: $5, $10, friend -> 2_2_1 (Configuration 7)
- TU_prompts[7]: $5, $10, stranger -> 2_2_2 (Configuration 8)
- TU_prompts[8]: $10, $5, friend -> 3_1_1  (Configuration 9)
- TU_prompts[9]: $10, $5, stranger -> 3_1_2 (Configuration 10)
- TU_prompts[10]: $10, $10, friend -> 3_2_1 (Configuration 11)
- TU_prompts[11]: $10, $10, stranger -> 3_2_2 (Configuration 12)

The results of the original experiment are:


| Configuration   | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   |
|------------------|------|------|------|------|------|------|------|------|------|------|------|------|
| $0              | 68%  | 6%   | 65%  | 6%   | 14%  | 0%   | 7%   | 0%   | 0%   | 0%   | 0%   | 0%   |
| $5              | 26%  | 77%  | 26%  | 16%  | 79%  | 79%  | 79%  | 14%  | 69%  | 42%  | 15%  | 0%   |
| $10             | 3%   | 10%  | 6%   | 58%  | 0%   | 7%   | 4%   | 57%  | 23%  | 46%  | 69%  | 73%  |
| Other          | 6%   | 6%   | 3%   | 19%  | 7%   | 14%  | 9%   | 29%  | 8%   | 12%  | 15%  | 27%  |
| N              | 31   | 31   | 31   | 31   | 28   | 28   | 28   | 28   | 26   | 26   | 26   | 26   |



In [11]:
og_results = {
    "og_results_1" : [68, 26, 3, 6],
    "og_results_2" : [6, 77, 10, 6],
    "og_results_3" : [65, 26, 6, 3],
    "og_results_4" : [6, 16, 58, 19],
    "og_results_5" : [14, 79, 7, 14],
    "og_results_6" : [0, 79, 7, 14],
    "og_results_7" : [7, 79, 4, 9],
    "og_results_8" : [0, 14, 57, 29],
    "og_results_9" : [0, 69, 23, 8],
    "og_results_10" : [0, 42, 46, 12],
    "og_results_11" : [0, 15, 69, 15],
    "og_results_12" : [0, 0, 73, 27]
}
    

- Setting up instructions the model should abide by

In [12]:
instructions = "Answer by only giving a single price in dollars and cents without an explanation."

In [13]:
# Gather all experiment IDs
experiment_ids =  ["TU_1_1_1_1", "TU_1_1_1_2", "TU_1_1_2_1", "TU_1_1_2_2", "TU_1_2_1_1", "TU_1_2_1_2", "TU_1_2_2_1", "TU_1_2_2_2", "TU_1_3_1_1", "TU_1_3_1_2", "TU_1_3_2_1", "TU_1_3_2_2",
                   "TU_2_1_1_1", "TU_2_1_1_2", "TU_2_1_2_1", "TU_2_1_2_2", "TU_2_2_1_1", "TU_2_2_1_2", "TU_2_2_2_1", "TU_2_2_2_2", "TU_2_3_1_1", "TU_2_3_1_2", "TU_2_3_2_1", "TU_2_3_2_2",
                   "TU_3_1_1_1", "TU_3_1_1_2", "TU_3_1_2_1", "TU_3_1_2_2", "TU_3_2_1_1", "TU_3_2_1_2", "TU_3_2_2_1", "TU_3_2_2_2", "TU_3_3_1_1", "TU_3_3_1_2", "TU_3_3_2_1", "TU_3_3_2_2"]

---

#### Dictionaries to extract information about the different experiments

In [14]:
# Dictionary to look up prompt for a given experiment id. key: experiment id, value: prompt
TU_experiment_prompts_dict = {
    "TU_1_1_1_1": TU_prompts[0],
    "TU_1_1_1_2": TU_prompts[1],
    "TU_1_1_2_1": TU_prompts[2],
    "TU_1_1_2_2": TU_prompts[3],
    "TU_1_2_1_1": TU_prompts[4],
    "TU_1_2_1_2": TU_prompts[5],
    "TU_1_2_2_1": TU_prompts[6],
    "TU_1_2_2_2": TU_prompts[7],
    "TU_1_3_1_1": TU_prompts[8],
    "TU_1_3_1_2": TU_prompts[9],
    "TU_1_3_2_1": TU_prompts[10],
    "TU_1_3_2_2": TU_prompts[11],
    "TU_2_1_1_1": TU_prompts[0],
    "TU_2_1_1_2": TU_prompts[1],
    "TU_2_1_2_1": TU_prompts[2],
    "TU_2_1_2_2": TU_prompts[3],
    "TU_2_2_1_1": TU_prompts[4],
    "TU_2_2_1_2": TU_prompts[5],
    "TU_2_2_2_1": TU_prompts[6],
    "TU_2_2_2_2": TU_prompts[7],
    "TU_2_3_1_1": TU_prompts[8],
    "TU_2_3_1_2": TU_prompts[9],
    "TU_2_3_2_1": TU_prompts[10],
    "TU_2_3_2_2": TU_prompts[11],
    "TU_3_1_1_1": TU_prompts[0],
    "TU_3_1_1_2": TU_prompts[1],
    "TU_3_1_2_1": TU_prompts[2],
    "TU_3_1_2_2": TU_prompts[3],
    "TU_3_2_1_1": TU_prompts[4],
    "TU_3_2_1_2": TU_prompts[5],
    "TU_3_2_2_1": TU_prompts[6],
    "TU_3_2_2_2": TU_prompts[7],
    "TU_3_3_1_1": TU_prompts[8],
    "TU_3_3_1_2": TU_prompts[9],
    "TU_3_3_2_1": TU_prompts[10],
    "TU_3_3_2_2": TU_prompts[11],
}

# Dictionary to look up which model to use for a given experiment id. key: experiment id, value: model name
TU_model_dict = {
    "TU_1_1_1_1": "gpt-3.5-turbo",
    "TU_1_1_1_2": "gpt-3.5-turbo",
    "TU_1_1_2_1": "gpt-3.5-turbo",
    "TU_1_1_2_2": "gpt-3.5-turbo",
    "TU_1_2_1_1": "gpt-3.5-turbo",
    "TU_1_2_1_2": "gpt-3.5-turbo",
    "TU_1_2_2_1": "gpt-3.5-turbo",
    "TU_1_2_2_2": "gpt-3.5-turbo",
    "TU_1_3_1_1": "gpt-3.5-turbo",
    "TU_1_3_1_2": "gpt-3.5-turbo",
    "TU_1_3_2_1": "gpt-3.5-turbo",
    "TU_1_3_2_2": "gpt-3.5-turbo",
    "TU_2_1_1_1": "gpt-4-1106-preview",
    "TU_2_1_1_2": "gpt-4-1106-preview",
    "TU_2_1_2_1": "gpt-4-1106-preview",
    "TU_2_1_2_2": "gpt-4-1106-preview",
    "TU_2_2_1_1": "gpt-4-1106-preview",
    "TU_2_2_1_2": "gpt-4-1106-preview",
    "TU_2_2_2_1": "gpt-4-1106-preview",
    "TU_2_2_2_2": "gpt-4-1106-preview",
    "TU_2_3_1_1": "gpt-4-1106-preview",
    "TU_2_3_1_2": "gpt-4-1106-preview",
    "TU_2_3_2_1": "gpt-4-1106-preview",
    "TU_2_3_2_2": "gpt-4-1106-preview",
    "TU_3_1_1_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU_3_1_1_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU_3_1_2_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU_3_1_2_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU_3_2_1_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU_3_2_1_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU_3_2_2_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU_3_2_2_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU_3_3_1_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU_3_3_1_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU_3_3_2_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU_3_3_2_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
}

# Dictionary to look up what prompt was used for a given experiment id. key: experiment id, value: prompt variable name
TU_prompt_ids_dict = {
    "TU_1_1_1_1": "TU_prompts[0]",
    "TU_1_1_1_2": "TU_prompts[1]",
    "TU_1_1_2_1": "TU_prompts[2]",
    "TU_1_1_2_2": "TU_prompts[3]",
    "TU_1_2_1_1": "TU_prompts[4]",
    "TU_1_2_1_2": "TU_prompts[5]",
    "TU_1_2_2_1": "TU_prompts[6]",
    "TU_1_2_2_2": "TU_prompts[7]",
    "TU_1_3_1_1": "TU_prompts[8]",
    "TU_1_3_1_2": "TU_prompts[9]",
    "TU_1_3_2_1": "TU_prompts[10]",
    "TU_1_3_2_2": "TU_prompts[11]",
    "TU_2_1_1_1": "TU_prompts[0]",
    "TU_2_1_1_2": "TU_prompts[1]",
    "TU_2_1_2_1": "TU_prompts[2]",
    "TU_2_1_2_2": "TU_prompts[3]",
    "TU_2_2_1_1": "TU_prompts[4]",
    "TU_2_2_1_2": "TU_prompts[5]",
    "TU_2_2_2_1": "TU_prompts[6]",
    "TU_2_2_2_2": "TU_prompts[7]",
    "TU_2_3_1_1": "TU_prompts[8]",
    "TU_2_3_1_2": "TU_prompts[9]",
    "TU_2_3_2_1": "TU_prompts[10]",
    "TU_2_3_2_2": "TU_prompts[11]",
    "TU_3_1_1_1": "TU_prompts[0]",
    "TU_3_1_1_2": "TU_prompts[1]",
    "TU_3_1_2_1": "TU_prompts[2]",
    "TU_3_1_2_2": "TU_prompts[3]",
    "TU_3_2_1_1": "TU_prompts[4]",
    "TU_3_2_1_2": "TU_prompts[5]",
    "TU_3_2_2_1": "TU_prompts[6]",
    "TU_3_2_2_2": "TU_prompts[7]",
    "TU_3_3_1_1": "TU_prompts[8]",
    "TU_3_3_1_2": "TU_prompts[9]",
    "TU_3_3_2_1": "TU_prompts[10]",
    "TU_3_3_2_2": "TU_prompts[11]",
    }

# Dictionary to look up initital ticket cotsts for a given experiment id. key: experiment id, value: initial costs
TU_initial_costs_dict = {
    "TU_1_1_1_1": 0,
    "TU_1_1_1_2": 0,
    "TU_1_1_2_1": 0,
    "TU_1_1_2_2": 0,
    "TU_1_2_1_1": 5,
    "TU_1_2_1_2": 5,
    "TU_1_2_2_1": 5,
    "TU_1_2_2_2": 5,
    "TU_1_3_1_1": 10,
    "TU_1_3_1_2": 10,
    "TU_1_3_2_1": 10,
    "TU_1_3_2_2": 10,
    "TU_2_1_1_1": 0,
    "TU_2_1_1_2": 0,
    "TU_2_1_2_1": 0,
    "TU_2_1_2_2": 0,
    "TU_2_2_1_1": 5,
    "TU_2_2_1_2": 5,
    "TU_2_2_2_1": 5,
    "TU_2_2_2_2": 5,
    "TU_2_3_1_1": 10,
    "TU_2_3_1_2": 10,
    "TU_2_3_2_1": 10,
    "TU_2_3_2_2": 10,
    "TU_3_1_1_1": 0,
    "TU_3_1_1_2": 0,
    "TU_3_1_2_1": 0,
    "TU_3_1_2_2": 0,
    "TU_3_2_1_1": 5,
    "TU_3_2_1_2": 5,
    "TU_3_2_2_1": 5,
    "TU_3_2_2_2": 5,
    "TU_3_3_1_1": 10,
    "TU_3_3_1_2": 10,
    "TU_3_3_2_1": 10,
    "TU_3_3_2_2": 10,
    }

# Dictionary to look up orientation prices for a given experiment id. key: experiment id, value: orientation price
TU_orientation_prices_dict = {
    "TU_1_1_1_1": 5,
    "TU_1_1_1_2": 5,
    "TU_1_1_2_1": 10,
    "TU_1_1_2_2": 10,
    "TU_1_2_1_1": 5,
    "TU_1_2_1_2": 5,
    "TU_1_2_2_1": 10,
    "TU_1_2_2_2": 10,
    "TU_1_3_1_1": 5,
    "TU_1_3_1_2": 5,
    "TU_1_3_2_1": 10,
    "TU_1_3_2_2": 10,
    "TU_2_1_1_1": 5,
    "TU_2_1_1_2": 5,
    "TU_2_1_2_1": 10,
    "TU_2_1_2_2": 10,
    "TU_2_2_1_1": 5,
    "TU_2_2_1_2": 5,
    "TU_2_2_2_1": 10,
    "TU_2_2_2_2": 10,
    "TU_2_3_1_1": 5,
    "TU_2_3_1_2": 5,
    "TU_2_3_2_1": 10,
    "TU_2_3_2_2": 10,
    "TU_3_1_1_1": 5,
    "TU_3_1_1_2": 5,
    "TU_3_1_2_1": 10,
    "TU_3_1_2_2": 10,
    "TU_3_2_1_1": 5,
    "TU_3_2_1_2": 5,
    "TU_3_2_2_1": 10,
    "TU_3_2_2_2": 10,
    "TU_3_3_1_1": 5,
    "TU_3_3_1_2": 5,
    "TU_3_3_2_1": 10,
    "TU_3_3_2_2": 10,
    }   

# Dictionary to look up potential buyers for a given experiment id. key: experiment id, value: potential buyer
TU_buyers_dict = {
    "TU_1_1_1_1": "friend",
    "TU_1_1_1_2": "stranger",
    "TU_1_1_2_1": "friend",
    "TU_1_1_2_2": "stranger",
    "TU_1_2_1_1": "friend",
    "TU_1_2_1_2": "stranger",
    "TU_1_2_2_1": "friend",
    "TU_1_2_2_2": "stranger",
    "TU_1_3_1_1": "friend",
    "TU_1_3_1_2": "stranger",
    "TU_1_3_2_1": "friend",
    "TU_1_3_2_2": "stranger",
    "TU_2_1_1_1": "friend",
    "TU_2_1_1_2": "stranger",
    "TU_2_1_2_1": "friend",
    "TU_2_1_2_2": "stranger",
    "TU_2_2_1_1": "friend",
    "TU_2_2_1_2": "stranger",
    "TU_2_2_2_1": "friend",
    "TU_2_2_2_2": "stranger",
    "TU_2_3_1_1": "friend",
    "TU_2_3_1_2": "stranger",
    "TU_2_3_2_1": "friend",
    "TU_2_3_2_2": "stranger",
    "TU_3_1_1_1": "friend",
    "TU_3_1_1_2": "stranger",
    "TU_3_1_2_1": "friend",
    "TU_3_1_2_2": "stranger",
    "TU_3_2_1_1": "friend",
    "TU_3_2_1_2": "stranger",
    "TU_3_2_2_1": "friend",
    "TU_3_2_2_2": "stranger",
    "TU_3_3_1_1": "friend",
    "TU_3_3_1_2": "stranger",
    "TU_3_3_2_1": "friend",
    "TU_3_3_2_2": "stranger",
}

# Dictionary to look up original results for a given experiment id. key: experiment id, value: original results
TU_results_dict = {
    "TU_1_1_1_1": [68, 26, 3, 6],
    "TU_1_1_1_2": [6, 77, 10, 6],
    "TU_1_1_2_1": [65, 26, 6, 3],
    "TU_1_1_2_2": [6, 16, 58, 19],
    "TU_1_2_1_1": [14, 79, 7, 14],
    "TU_1_2_1_2": [0, 79, 7, 14],
    "TU_1_2_2_1": [7, 79, 4, 9],
    "TU_1_2_2_2": [0, 14, 57, 29],
    "TU_1_3_1_1": [0, 69, 23, 8],
    "TU_1_3_1_2": [0, 42, 46, 12],
    "TU_1_3_2_1": [0, 15, 69, 15],
    "TU_1_3_2_2": [0, 0, 73, 27],
    "TU_2_1_1_1": [68, 26, 3, 6],
    "TU_2_1_1_2": [6, 77, 10, 6],
    "TU_2_1_2_1": [65, 26, 6, 3],
    "TU_2_1_2_2": [6, 16, 58, 19],
    "TU_2_2_1_1": [14, 79, 7, 14],
    "TU_2_2_1_2": [0, 79, 7, 14],
    "TU_2_2_2_1": [7, 79, 4, 9],
    "TU_2_2_2_2": [0, 14, 57, 29],
    "TU_2_3_1_1": [0, 69, 23, 8],
    "TU_2_3_1_2": [0, 42, 46, 12],
    "TU_2_3_2_1": [0, 15, 69, 15],
    "TU_2_3_2_2": [0, 0, 73, 27],
    "TU_3_1_1_1": [68, 26, 3, 6],
    "TU_3_1_1_2": [6, 77, 10, 6],
    "TU_3_1_2_1": [65, 26, 6, 3],
    "TU_3_1_2_2": [6, 16, 58, 19],
    "TU_3_2_1_1": [14, 79, 7, 14],
    "TU_3_2_1_2": [0, 79, 7, 14],
    "TU_3_2_2_1": [7, 79, 4, 9],
    "TU_3_2_2_2": [0, 14, 57, 29],
    "TU_3_3_1_1": [0, 69, 23, 8],
    "TU_3_3_1_2": [0, 42, 46, 12],
    "TU_3_3_2_1": [0, 15, 69, 15],
    "TU_3_3_2_2": [0, 0, 73, 27],
}

# Dictionary to look up number of original answers. key: experiment id, value: number of original answers
TU_answercount_dict = {
    "TU_1_1_1_1": 31,
    "TU_1_1_1_2": 31,
    "TU_1_1_2_1": 31,
    "TU_1_1_2_2": 31,
    "TU_1_2_1_1": 28,
    "TU_1_2_1_2": 28,
    "TU_1_2_2_1": 28,
    "TU_1_2_2_2": 28,
    "TU_1_3_1_1": 26,
    "TU_1_3_1_2": 26,
    "TU_1_3_2_1": 26,
    "TU_1_3_2_2": 26,
    "TU_2_1_1_1": 31,
    "TU_2_1_1_2": 31,
    "TU_2_1_2_1": 31,
    "TU_2_1_2_2": 31,
    "TU_2_2_1_1": 28,
    "TU_2_2_1_2": 28,
    "TU_2_2_2_1": 28,
    "TU_2_2_2_2": 28,
    "TU_2_3_1_1": 26,
    "TU_2_3_1_2": 26,
    "TU_2_3_2_1": 26,
    "TU_2_3_2_2": 26,
    "TU_3_1_1_1": 31,
    "TU_3_1_1_2": 31,
    "TU_3_1_2_1": 31,
    "TU_3_1_2_2": 31,
    "TU_3_2_1_1": 28,
    "TU_3_2_1_2": 28,
    "TU_3_2_2_1": 28,
    "TU_3_2_2_2": 28,
    "TU_3_3_1_1": 26,
    "TU_3_3_1_2": 26,
    "TU_3_3_2_1": 26,
    "TU_3_3_2_2": 26
}

# Dictionary to look configuration given experiment id. key: experiment id, value: configuration (0-12)
TU_configurations_dict = {
    "TU_1_1_1_1": 1,
    "TU_1_1_1_2": 2,
    "TU_1_1_2_1": 3,
    "TU_1_1_2_2": 4,
    "TU_1_2_1_1": 5,
    "TU_1_2_1_2": 6,
    "TU_1_2_2_1": 7,
    "TU_1_2_2_2": 8,
    "TU_1_3_1_1": 9,
    "TU_1_3_1_2": 10,
    "TU_1_3_2_1": 11,
    "TU_1_3_2_2": 12,
    "TU_2_1_1_1": 1,
    "TU_2_1_1_2": 2,
    "TU_2_1_2_1": 3,
    "TU_2_1_2_2": 4,
    "TU_2_2_1_1": 5,
    "TU_2_2_1_2": 6,
    "TU_2_2_2_1": 7,
    "TU_2_2_2_2": 8,
    "TU_2_3_1_1": 9,
    "TU_2_3_1_2": 10,
    "TU_2_3_2_1": 11,
    "TU_2_3_2_2": 12,
    "TU_3_1_1_1": 1,
    "TU_3_1_1_2": 2,
    "TU_3_1_2_1": 3,
    "TU_3_1_2_2": 4,
    "TU_3_2_1_1": 5,
    "TU_3_2_1_2": 6,
    "TU_3_2_2_1": 7,
    "TU_3_2_2_2": 8,
    "TU_3_3_1_1": 9,
    "TU_3_3_1_2": 10,
    "TU_3_3_2_1": 11,
    "TU_3_3_2_2": 10
}

# Dictionary to look up experiment id as string for a given experiment id. key: experiment id, value: experiment id as string
TU_experiment_ids_dict = {
    "TU_1_1_1_1": "TU_1_1_1_1",
    "TU_1_1_1_2": "TU_1_1_1_2",
    "TU_1_1_2_1": "TU_1_1_2_1",
    "TU_1_1_2_2": "TU_1_1_2_2",
    "TU_1_2_1_1": "TU_1_2_1_1",
    "TU_1_2_1_2": "TU_1_2_1_2",
    "TU_1_2_2_1": "TU_1_2_2_1",
    "TU_1_2_2_2": "TU_1_2_2_2",
    "TU_1_3_1_1": "TU_1_3_1_1",
    "TU_1_3_1_2": "TU_1_3_1_2",
    "TU_1_3_2_1": "TU_1_3_2_1",
    "TU_1_3_2_2": "TU_1_3_2_2",
    "TU_2_1_1_1": "TU_2_1_1_1",
    "TU_2_1_1_2": "TU_2_1_1_2",
    "TU_2_1_2_1": "TU_2_1_2_1",
    "TU_2_1_2_2": "TU_2_1_2_2",
    "TU_2_2_1_1": "TU_2_2_1_1",
    "TU_2_2_1_2": "TU_2_2_1_2",
    "TU_2_2_2_1": "TU_2_2_2_1",
    "TU_2_2_2_2": "TU_2_2_2_2",
    "TU_2_3_1_1": "TU_2_3_1_1",
    "TU_2_3_1_2": "TU_2_3_1_2",
    "TU_2_3_2_1": "TU_2_3_2_1",
    "TU_2_3_2_2": "TU_2_3_2_2",
    "TU_3_1_1_1": "TU_3_1_1_1",
    "TU_3_1_1_2": "TU_3_1_1_2",
    "TU_3_1_2_1": "TU_3_1_2_1",
    "TU_3_1_2_2": "TU_3_1_2_2",
    "TU_3_2_1_1": "TU_3_2_1_1",
    "TU_3_2_1_2": "TU_3_2_1_2",
    "TU_3_2_2_1": "TU_3_2_2_1",
    "TU_3_2_2_2": "TU_3_2_2_2",
    "TU_3_3_1_1": "TU_3_3_1_1",
    "TU_3_3_1_2": "TU_3_3_1_2",
    "TU_3_3_2_1": "TU_3_3_2_1",
    "TU_3_3_2_2": "TU_3_3_2_2",
}


---

#### Setting up functions to repeatedly prompt the LLMs

- Helper function to extract dollar amount of given answers

In [15]:
def extract_dollar_amounts(answers):
    # Only return values that start with "$"
    valid_prices = [item for item in answers if item.startswith("$") and item[1:].replace(',', '').replace('.', '').isdigit()] # check if everything after $ is a digit, exlcuding commas
    # Delete the "$" from the beginning of each price
    prices = [item.replace('$', '') for item in valid_prices]
    return prices

- Functions to query 1 prompt n times for OpenAI Models

In [16]:
def TU_run_experiment(experiment_id, n, progress_bar, temperature):
    
    answers = []
    for _ in range(n): 
        response = client.chat.completions.create(
            model = TU_model_dict[experiment_id], 
            max_tokens = 2,
            temperature = temperature, # range is 0 to 2
            messages = [
            {"role": "system", "content": "Answer by only giving a single price in dollars and cents without an explanation."},        
            {"role": "user", "content": 
             f"{TU_experiment_prompts_dict[experiment_id]} Answer by only giving a single price in dollars and cents without an explanation."}
                   ])

        # Store the answer in the list
        answer = response.choices[0].message.content
        answers.append(answer.strip())
        # Update progress bar (given from either temperature loop, or set locally)
        progress_bar.update(1)

    # Extract valid prices from answers
    valid_prices = extract_dollar_amounts(answers)

    # Compute number of valid answers
    n_observations = len(valid_prices)

    # Collect results 
    results = [experiment_id, temperature, TU_model_dict[experiment_id], TU_initial_costs_dict[experiment_id], TU_orientation_prices_dict[experiment_id],
                TU_buyers_dict[experiment_id], answers, n_observations, TU_configurations_dict[experiment_id], TU_results_dict[experiment_id], TU_answercount_dict[experiment_id]]

    # Give out results
    return results

- Functions to query 1 prompt n times (LLama)

In [17]:
def TU_run_experiment_llama(experiment_id, n, progress_bar, temperature):
    answers = []
    for _ in range(n):
        response = replicate.run(
            TU_model_dict[experiment_id],
            input = {
                "system_prompt":  "Answer by only giving a single price in dollars and cents without an explanation.",
                "temperature": temperature,
                "max_new_tokens": 10, 
                "prompt": f"{TU_experiment_prompts_dict[experiment_id]} Answer by only giving a single price in dollars and cents without an explanation."
            }
        )
        # Grab answer and append to list
        answer = "" # Set to empty string, otherwise it would append the previous answer to the new one
        for item in response:
            answer = answer + item
        answers.append(answer.strip())

        # Update progress bar
        progress_bar.update(1)

    
    # Extract valid prices from answers
    valid_prices = extract_dollar_amounts(answers)

    # Compute number of valid answers
    n_observations = len(valid_prices)

    # Collect results 
    results = [experiment_id, temperature, TU_model_dict[experiment_id], TU_initial_costs_dict[experiment_id], TU_orientation_prices_dict[experiment_id],
               TU_buyers_dict[experiment_id], answers, n_observations, TU_configurations_dict[experiment_id], TU_results_dict[experiment_id], TU_answercount_dict[experiment_id]]
    
    # Give out results
    return results

- Function to loop run_experiment() over a list of temperature values

In [18]:
def TU_temperature_loop(function, experiment_id, temperature_list = [0.5, 1, 1.5], n = 50):
    """
    Function to run an experiment with different temperature values.
    
    Args:
        function (function): Function to be used for querying ChatGPT i.e. run_experiment()
        experiment_id (str): ID of th e experiment to be run. Contains info about prompt and model
        temperature_list (list): List of temperature values to be looped over
        n: Number of requests for each prompt per temperature value
        max_tokens: Maximum number of tokens in response object
        
    Returns:
        results_df: Dataframe with experiment results
        probs_df: Dataframe with answer probabilities
    """    
    # Empty list for storing results
    results_list = []

    # Initialize progress bar -> used as input for run_experiment()
    progress_bar = tqdm(range(n*len(temperature_list)))

    # Loop over different temperature values, calling the input function n times each (i.e. queriyng ChatGPT n times)
    for temperature in temperature_list:
        results = function(experiment_id = experiment_id, n = n, temperature = temperature, progress_bar = progress_bar) 
        results_list.append(results)
       

    # Horizontally concatenate the results, transpose, and set index
    results_df = pd.DataFrame(results_list).transpose().set_index(pd.Index(
        ["experiment_id", "temperature", "model", "initial_cost", "orientation_price", "buyer", "answers", "Obs.", "configuration", "original", "original_count"]))
  
   
    # Return some information about the experiment as a check
    check = f"In this run, a total of {n*len(temperature_list)} requests were made using {TU_prompt_ids_dict[experiment_id]}."
    # Print information about the experiment
    print(check)
 
    return results_df

---

### Model 1: GPT-3.5-Turbo

In [90]:
# For GPT-3.5-turbo we make 100 requests per prompt & temperature value
N = 100
# List of temperature values to loop over
temperature_list = [0.01, 0.5, 1, 1.5, 2]

- Configuration 1

In [91]:
results_1 = []
results_1_1 = TU_temperature_loop(TU_run_experiment, "TU_1_1_1_1", temperature_list = temperature_list, n = N)
results_1.append(results_1_1)

100%|██████████| 500/500 [14:10<00:00,  1.70s/it]   

In this run, a total of 500 requests were made using TU_prompts[0].


- Configuration 2

In [92]:
results_2 = []
results_2_1 = TU_temperature_loop(TU_run_experiment, "TU_1_1_1_2", temperature_list = temperature_list, n = N)
results_2.append(results_2_1)

100%|██████████| 500/500 [03:41<00:00,  2.26it/s]

In this run, a total of 500 requests were made using TU_prompts[1].


- Configuration 3

In [93]:
results_3 = []
results_3_1 = TU_temperature_loop(TU_run_experiment, "TU_1_1_2_1", temperature_list = temperature_list, n = N)
results_3.append(results_3_1)

100%|██████████| 500/500 [03:38<00:00,  2.28it/s]

In this run, a total of 500 requests were made using TU_prompts[2].


- Configuration 4

In [94]:
results_4 = []
results_4_1 = TU_temperature_loop(TU_run_experiment, "TU_1_1_2_2", temperature_list = temperature_list, n = N)
results_4.append(results_4_1)

100%|██████████| 500/500 [03:42<00:00,  2.24it/s]

In this run, a total of 500 requests were made using TU_prompts[3].


- Configuration 5

In [95]:
results_5 = []
results_5_1 = TU_temperature_loop(TU_run_experiment, "TU_1_2_1_1", temperature_list = temperature_list, n = N)
results_5.append(results_5_1)

100%|██████████| 500/500 [03:54<00:00,  2.13it/s]

In this run, a total of 500 requests were made using TU_prompts[4].


- Configuration 6

In [96]:
results_6 = []
results_6_1 = TU_temperature_loop(TU_run_experiment, "TU_1_2_1_2", temperature_list = temperature_list, n = N)
results_6.append(results_6_1)

100%|██████████| 500/500 [03:37<00:00,  2.30it/s]

In this run, a total of 500 requests were made using TU_prompts[5].


- Configuration 7

In [97]:
results_7 = []
results_7_1 = TU_temperature_loop(TU_run_experiment, "TU_1_2_2_1", temperature_list = temperature_list, n = N)
results_7.append(results_7_1)

100%|██████████| 500/500 [13:45<00:00,  1.65s/it]   

In this run, a total of 500 requests were made using TU_prompts[6].


- Configuration 8

In [98]:
results_8 = []
results_8_1 = TU_temperature_loop(TU_run_experiment, "TU_1_2_2_2", temperature_list = temperature_list, n = N)
results_8.append(results_8_1)

100%|██████████| 500/500 [03:41<00:00,  2.26it/s]

In this run, a total of 500 requests were made using TU_prompts[7].


- Configuration 9

In [99]:
results_9 = []
results_9_1 = TU_temperature_loop(TU_run_experiment, "TU_1_3_1_1", temperature_list = temperature_list, n = N)
results_9.append(results_9_1)

100%|██████████| 500/500 [03:39<00:00,  2.27it/s]

In this run, a total of 500 requests were made using TU_prompts[8].


- Configuration 10

In [100]:
results_10 = []
results_10_1 = TU_temperature_loop(TU_run_experiment, "TU_1_3_1_2", temperature_list = temperature_list, n = N)
results_10.append(results_10_1)

100%|██████████| 500/500 [03:47<00:00,  2.20it/s]

In this run, a total of 500 requests were made using TU_prompts[9].


- Configuration 11

In [101]:
results_11 = []
results_11_1 = TU_temperature_loop(TU_run_experiment, "TU_1_3_2_1", temperature_list = temperature_list, n = N)
results_11.append(results_11_1)

100%|██████████| 500/500 [03:37<00:00,  2.30it/s]

In this run, a total of 500 requests were made using TU_prompts[10].


- Configuration 12

In [102]:
results_12 = []
results_12_1 = TU_temperature_loop(TU_run_experiment, "TU_1_3_2_2", temperature_list = temperature_list, n = N)
results_12.append(results_12_1)

100%|██████████| 500/500 [23:45<00:00,  2.85s/it]   

In this run, a total of 500 requests were made using TU_prompts[11].


---

### Model 2: GPT-4-1106-Preview

In [103]:
N = 50

- Configuration 1

In [104]:
results_1_2 = TU_temperature_loop(TU_run_experiment, "TU_2_1_1_1", temperature_list = temperature_list, n = N)
results_1.append(results_1_2)

100%|██████████| 250/250 [03:40<00:00,  1.14it/s]

In this run, a total of 250 requests were made using TU_prompts[0].


- Configuration 2

In [105]:
results_2_2 = TU_temperature_loop(TU_run_experiment, "TU_2_1_1_2", temperature_list = temperature_list, n = N)
results_2.append(results_2_2)

100%|██████████| 250/250 [03:39<00:00,  1.14it/s]

In this run, a total of 250 requests were made using TU_prompts[1].


- Configuration 3

In [106]:
results_3_2 = TU_temperature_loop(TU_run_experiment, "TU_2_1_2_1", temperature_list = temperature_list, n = N)
results_3.append(results_3_2)

100%|██████████| 250/250 [03:19<00:00,  1.25it/s]

In this run, a total of 250 requests were made using TU_prompts[2].


- Configuration 4

In [107]:
results_4_2 = TU_temperature_loop(TU_run_experiment, "TU_2_1_2_2", temperature_list = temperature_list, n = N)
results_4.append(results_4_2)

100%|██████████| 250/250 [23:19<00:00,  5.60s/it]   

In this run, a total of 250 requests were made using TU_prompts[3].


- Configuration 5

In [108]:
results_5_2 = TU_temperature_loop(TU_run_experiment, "TU_2_2_1_1", temperature_list = temperature_list, n = N)
results_5.append(results_5_2)

100%|██████████| 250/250 [03:10<00:00,  1.32it/s]

In this run, a total of 250 requests were made using TU_prompts[4].


- Configuration 6

In [109]:
results_6_2 = TU_temperature_loop(TU_run_experiment, "TU_2_2_1_2", temperature_list = temperature_list, n = N)
results_6.append(results_6_2)

100%|██████████| 250/250 [04:51<00:00,  1.17s/it]

In this run, a total of 250 requests were made using TU_prompts[5].


- Configuration 7

In [110]:
results_7_2 = TU_temperature_loop(TU_run_experiment, "TU_2_2_2_1", temperature_list = temperature_list, n = N)
results_7.append(results_7_2)

100%|██████████| 250/250 [13:03<00:00,  3.13s/it]  

In this run, a total of 250 requests were made using TU_prompts[6].


- Configuration 8

In [111]:
results_8_2 = TU_temperature_loop(TU_run_experiment, "TU_2_2_2_2", temperature_list = temperature_list, n = N)
results_8.append(results_8_2)

100%|██████████| 250/250 [03:04<00:00,  1.35it/s]

In this run, a total of 250 requests were made using TU_prompts[7].


- Configuration 9

In [112]:
results_9_2 = TU_temperature_loop(TU_run_experiment, "TU_2_3_1_1", temperature_list = temperature_list, n = N)
results_9.append(results_9_2)

100%|██████████| 250/250 [03:12<00:00,  1.30it/s]

In this run, a total of 250 requests were made using TU_prompts[8].


- Configuration 10

In [113]:
results_10_2 = TU_temperature_loop(TU_run_experiment, "TU_2_3_1_2", temperature_list = temperature_list, n = N)
results_10.append(results_10_2)

100%|██████████| 250/250 [03:34<00:00,  1.17it/s]

In this run, a total of 250 requests were made using TU_prompts[9].


- Configuration 11

In [114]:
results_11_2 = TU_temperature_loop(TU_run_experiment, "TU_2_3_2_1", temperature_list = temperature_list, n = N)
results_11.append(results_11_2)

100%|██████████| 250/250 [03:11<00:00,  1.31it/s]

In this run, a total of 250 requests were made using TU_prompts[10].


- Configuration 12

In [115]:
results_12_2 = TU_temperature_loop(TU_run_experiment, "TU_2_3_2_2", temperature_list = temperature_list, n = N)
results_12.append(results_12_2)

100%|██████████| 250/250 [03:03<00:00,  1.36it/s]

In this run, a total of 250 requests were made using TU_prompts[11].


---

### Model 3: LLama-2-70b

In [118]:
N = 50

- Configuration 1

In [119]:
results_1_3 = TU_temperature_loop(TU_run_experiment_llama, "TU_3_1_1_1", temperature_list = temperature_list, n = N)
results_1.append(results_1_3)

100%|██████████| 250/250 [07:33<00:00,  1.81s/it] 

In this run, a total of 250 requests were made using TU_prompts[0].


- Configuration 2

In [120]:
results_2_3 = TU_temperature_loop(TU_run_experiment_llama, "TU_3_1_1_2", temperature_list = temperature_list, n = N)
results_2.append(results_2_3)

100%|██████████| 250/250 [09:06<00:00,  2.18s/it]

In this run, a total of 250 requests were made using TU_prompts[1].


- Configuration 3

In [121]:
results_3_3 = TU_temperature_loop(TU_run_experiment_llama, "TU_3_1_2_1", temperature_list = temperature_list, n = N)
results_3.append(results_3_3)

100%|██████████| 250/250 [05:41<00:00,  1.37s/it]

In this run, a total of 250 requests were made using TU_prompts[2].


- Configuration 4

In [122]:
results_4_3 = TU_temperature_loop(TU_run_experiment_llama, "TU_3_1_2_2", temperature_list = temperature_list, n = N)
results_4.append(results_4_3)

100%|██████████| 250/250 [08:28<00:00,  2.03s/it] 

In this run, a total of 250 requests were made using TU_prompts[3].


- Configuration 5

In [123]:
results_5_3 = TU_temperature_loop(TU_run_experiment_llama, "TU_3_2_1_1", temperature_list = temperature_list, n = N)
results_5.append(results_5_3)

100%|██████████| 250/250 [06:22<00:00,  1.53s/it]

In this run, a total of 250 requests were made using TU_prompts[4].


- Configuration 6

In [124]:
results_6_3 = TU_temperature_loop(TU_run_experiment_llama, "TU_3_2_1_2", temperature_list = temperature_list, n = N)
results_6.append(results_6_3)

100%|██████████| 250/250 [05:41<00:00,  1.37s/it]

In this run, a total of 250 requests were made using TU_prompts[5].


- Configuration 7

In [125]:
results_7_3 = TU_temperature_loop(TU_run_experiment_llama, "TU_3_2_2_1", temperature_list = temperature_list, n = N)
results_7.append(results_7_3)

100%|██████████| 250/250 [06:07<00:00,  1.47s/it]

In this run, a total of 250 requests were made using TU_prompts[6].


- Configuration 8

In [126]:
results_8_3 = TU_temperature_loop(TU_run_experiment_llama, "TU_3_2_2_2", temperature_list = temperature_list, n = N)
results_8.append(results_8_3)

100%|██████████| 250/250 [05:01<00:00,  1.20s/it]

In this run, a total of 250 requests were made using TU_prompts[7].


- Configuration 9

In [127]:
results_9_3 = TU_temperature_loop(TU_run_experiment_llama, "TU_3_3_1_1", temperature_list = temperature_list, n = N)
results_9.append(results_9_3)

100%|██████████| 250/250 [05:04<00:00,  1.22s/it]

In this run, a total of 250 requests were made using TU_prompts[8].


- Configuration 10

In [128]:
results_10_3 = TU_temperature_loop(TU_run_experiment_llama, "TU_3_3_1_2", temperature_list = temperature_list, n = N)
results_10.append(results_10_3)

100%|██████████| 250/250 [07:49<00:00,  1.88s/it] 

In this run, a total of 250 requests were made using TU_prompts[9].


- Configuration 11

In [129]:
results_11_3 = TU_temperature_loop(TU_run_experiment_llama, "TU_3_3_2_1", temperature_list = temperature_list, n = N)
results_11.append(results_11_3)

100%|██████████| 250/250 [05:46<00:00,  1.39s/it]

In this run, a total of 250 requests were made using TU_prompts[10].


- Configuration 12

In [130]:
results_12_3 = TU_temperature_loop(TU_run_experiment_llama, "TU_3_3_2_2", temperature_list = temperature_list, n = N)
results_12.append(results_12_3)

100%|██████████| 250/250 [07:45<00:00,  1.86s/it]

In this run, a total of 250 requests were made using TU_prompts[11].


----

#### Gather all results and save to .csv

In [131]:
# Concatenate results
results_1_df = pd.concat(results_1, axis = 1).transpose()
results_2_df = pd.concat(results_2, axis = 1).transpose()
results_3_df = pd.concat(results_3, axis = 1).transpose()
results_4_df = pd.concat(results_4, axis = 1).transpose()
results_5_df = pd.concat(results_5, axis = 1).transpose()
results_6_df = pd.concat(results_6, axis = 1).transpose()
results_7_df = pd.concat(results_7, axis = 1).transpose()
results_8_df = pd.concat(results_8, axis = 1).transpose()
results_9_df = pd.concat(results_9, axis = 1).transpose()
results_10_df = pd.concat(results_10, axis = 1).transpose()
results_11_df = pd.concat(results_11, axis = 1).transpose()
results_12_df = pd.concat(results_12, axis = 1).transpose()

# Concatenate all results
TU_results = pd.concat([results_1_df, results_2_df, results_3_df, results_4_df, results_5_df,
                         results_6_df, results_7_df, results_8_df, results_9_df, results_10_df, results_11_df, results_12_df], axis = 0)

# Rename LLama model
TU_results['model'] = TU_results['model'].replace('meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3', 
                                  'llama-2-70b')

# Capitalize first letter of column names
TU_results.columns = TU_results.columns.str.capitalize()

# Display results
TU_results

,Experiment_id,Temperature,Model,Initial_cost,Orientation_price,Buyer,Answers,Obs.,Configuration,Original,Original_count
0,TU_1_1_1_1,0.01,gpt-3.5-turbo,0,5,friend,"[$5, $5, $5, $5, $5, $5, $5, $5, $5, $5, $5, $...",100,1,"[68, 26, 3, 6]",31
1,TU_1_1_1_1,0.5,gpt-3.5-turbo,0,5,friend,"[$5, $5, $5, $5, $5, $5, $5, $5, $5, $5, $5, $...",100,1,"[68, 26, 3, 6]",31
2,TU_1_1_1_1,1.0,gpt-3.5-turbo,0,5,friend,"[$5, $5, $5, $5, $5, $5, $5, $5, $5, $5, $5, $...",100,1,"[68, 26, 3, 6]",31
3,TU_1_1_1_1,1.5,gpt-3.5-turbo,0,5,friend,"[$5, $5, $5, $10, The price, $5, $5, $5, $5, $...",95,1,"[68, 26, 3, 6]",31
4,TU_1_1_1_1,2.0,gpt-3.5-turbo,0,5,friend,"[$5, $5, 98 dollars, Can |, Technical tools, $...",68,1,"[68, 26, 3, 6]",31
...,...,...,...,...,...,...,...,...,...,...,...
0,TU_3_3_2_2,0.01,llama-2-70b,10,10,stranger,"[$15, $15, $15, $15, $15, $15, $15, $15, $15, ...",50,10,"[0, 0, 73, 27]",26
1,TU_3_3_2_2,0.5,llama-2-70b,10,10,stranger,"[$15, $15, $15, $15, $15, $15, $15, $15, $15, ...",50,10,"[0, 0, 73, 27]",26
2,TU_3_3_2_2,1.0,llama-2-70b,10,10,stranger,"[$15, $15, $15, $15, $15, $15, $15, $15, $15, ...",50,10,"[0, 0, 73, 27]",26
3,TU_3_3_2_2,1.5,llama-2-70b,10,10,stranger,"[$15, $15, $15, $15, $15, $15, $15, $15, $15, ...",50,10,"[0, 0, 73, 27]",26


In [132]:
# Save results
TU_results.to_csv("Output/TU_results.csv", index = False)

---

### Visualization of results

In [19]:
TU_results = pd.read_csv("Output/TU_results.csv")

In [20]:
def TU_plot_results(model, temperature, initial_cost, orientation_price, buyer, df):
    # Select requested subset of results
    df = df[(df["Model"] == model) & (df["Temperature"] == temperature) & (df["Initial_cost"] == initial_cost) & (df["Orientation_price"] == orientation_price) & (df["Buyer"] == buyer)]
    # Transpose for plotting
    df = df.transpose()
    # Get original and model answers
    og_answers = df.loc["Original"].apply(literal_eval).iloc[0]
    # Apply literal_eval to convert string to list
    answers = df.loc["Answers"].apply(literal_eval).iloc[0]
    # Get number of observations 
    n_observations = df.loc["Obs."].iloc[0] 
    # Get number of original answers
    n_original = df.loc["Original_count"].iloc[0]
    # Get stated WTP
    prices = extract_dollar_amounts(answers)
    # Get temperature value 
    temperature = temperature

    # Compute percentage of $0:
    percent_0 = (prices.count("0")/n_observations)*100
    # Compute percentage of $5:
    percent_5 = (prices.count("5")/n_observations)*100
    # Compute percentage of $10:
    percent_10 = (prices.count("10")/n_observations)*100
    # Compute percentage of $15:
    percent_15 = (prices.count("15")/n_observations)*100
    # Compute percentage of other answers:
    percent_other = 100-percent_0-percent_5-percent_10-percent_15

    fig = go.Figure(data = [
        go.Bar(
            name = "Model answers",
            x = ["$0", "$5", "$10", "$15", "Other"],
            y = [percent_0, percent_5, percent_10, percent_15, percent_other],
            customdata = [n_observations, n_observations, n_observations, n_observations, n_observations], 
            hovertemplate = "Percentage: %{y:.2f}%<br>Number of observations: %{customdata}<extra></extra>",
            marker_color = "rgb(55, 83, 109)"
        ),
        go.Bar(
            name = "Original answers",
            x = ["$0", "$5", "$10", "$15", "Other"],
            y = [og_answers[0], og_answers[1], og_answers[2], 0, og_answers[3]], # 0 because no-one answered $15, but model did 
            customdata = [n_original, n_original, n_original, n_original, n_original],
            hovertemplate = "Percentage: %{y:.2f}%<br>Number of observations: %{customdata}<extra></extra>",
            marker_color = "rgb(26, 118, 255)"
        )
    ])

    # Style figure and add labels 
    fig.update_layout(
    barmode = "group",
     xaxis = dict(
            title = "Price",
            titlefont_size = 18,
            tickfont_size = 16,
     ),
    yaxis = dict(
        title = "Percentage",
        titlefont_size = 18,
        tickfont_size = 16,
    ),
    title = dict(
    text =  f"Distribution of answers for temperature {temperature}, using model {model}",
    x = 0.5, 
    y = 0.95,
    font_size = 18,
    ),
    legend=dict(
        x=1.01,  # Adjust the position of the legend
        y=0.9,
        font=dict(family='Arial', size=12, color='black'),
        bordercolor='black',  # Set the legend border color
        borderwidth=2,  # Set the legend border width
    ),
    width = 1000,
    margin=dict(t=60)
    )
    
    fig.show()

In [25]:
df = TU_results
model = "llama-2-70b"
temperature = 2
initial_costs = 5
orientation_price = 10
buyer = "friend"
TU_plot_results(model, temperature, initial_costs, orientation_price, buyer, df)


---